In [2]:
from src.database_config.database import engine
from src.models.prices_latest import PriceLatest
from sqlalchemy.orm import Session

from sqlalchemy import text
session = Session(engine)

with engine.connect() as conn:
    x = (session.query(PriceLatest)
            .filter_by(article_id=100)
            .first())
    print("f")



2025-10-11 15:41:26,198 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-10-11 15:41:26,199 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-11 15:41:26,203 INFO sqlalchemy.engine.Engine select current_schema()
2025-10-11 15:41:26,204 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-11 15:41:26,205 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-10-11 15:41:26,205 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-11 15:41:26,236 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 15:41:26,238 INFO sqlalchemy.engine.Engine SELECT prices_latest.id AS prices_latest_id, prices_latest.article_id AS prices_latest_article_id, prices_latest.price AS prices_latest_price, prices_latest.price_per_unit AS prices_latest_price_per_unit, prices_latest.price_per_unit_base AS prices_latest_price_per_unit_base, prices_latest.timestamp AS prices_latest_timestamp 
FROM prices_latest 
WHERE prices_latest.article_id = %(article_id_1)s 
 LIMIT %(param_1)s
2025-10-

In [3]:
from src.database_config.database import engine
from src.models.prices_latest import PriceLatest
from sqlalchemy.orm import Session

session = Session(engine)


# Query the row
x = session.query(PriceLatest).filter_by(article_id=100).first()

# Nice print
if x:
    print(f"PriceLatest[article_id={x.article_id}]: id={x.id}, price={x.price}, price_per_unit={x.price_per_unit}, timestamp={x.timestamp}")
else:
    print("No PriceLatest found for article_id=100")


2025-10-11 15:42:05,080 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 15:42:05,082 INFO sqlalchemy.engine.Engine SELECT prices_latest.id AS prices_latest_id, prices_latest.article_id AS prices_latest_article_id, prices_latest.price AS prices_latest_price, prices_latest.price_per_unit AS prices_latest_price_per_unit, prices_latest.price_per_unit_base AS prices_latest_price_per_unit_base, prices_latest.timestamp AS prices_latest_timestamp 
FROM prices_latest 
WHERE prices_latest.article_id = %(article_id_1)s 
 LIMIT %(param_1)s
2025-10-11 15:42:05,082 INFO sqlalchemy.engine.Engine [cached since 38.84s ago] {'article_id_1': 100, 'param_1': 1}
PriceLatest[article_id=100]: id=11256, price=3.79, price_per_unit=0.32, timestamp=2025-10-10


In [10]:
if (
        latest_price := session.query(PriceLatest)
        .filter_by(article_id=article.id)
        .first()
) and (
    latest_price.price == 2.39
) and (
    latest_price.price_per_unit == 0.24
    ):
        print(
            f"Unchanged price for article: {3}."
        )
else: 
    if not latest_price:
        latest_price = PriceLatest(article=article)
        session.add(latest_price)
    print(
        f"New prices for article: {3}."
    )

2025-10-11 17:14:35,848 INFO sqlalchemy.engine.Engine SELECT prices_latest.id AS prices_latest_id, prices_latest.article_id AS prices_latest_article_id, prices_latest.price AS prices_latest_price, prices_latest.price_per_unit AS prices_latest_price_per_unit, prices_latest.price_per_unit_base AS prices_latest_price_per_unit_base, prices_latest.timestamp AS prices_latest_timestamp 
FROM prices_latest 
WHERE prices_latest.article_id = %(article_id_1)s 
 LIMIT %(param_1)s
2025-10-11 17:14:35,848 INFO sqlalchemy.engine.Engine [cached since 2454s ago] {'article_id_1': 4, 'param_1': 1}
Unchanged price for article: 3.


In [15]:
from decimal import Decimal
repr(Decimal(13.393))

"Decimal('13.393000000000000682121026329696178436279296875')"